In [2]:
# ----------------------------------
# Implementation
# ----------------------------------

using Distributed
nprocs() == 1 && addprocs(; exeflags = "--project")
workers()

@everywhere using AutoMLPipeline
@everywhere using DataFrames
@everywhere using AutoMLPipeline: score
@everywhere using Random
ENV["COLUMNS"]=1000
ENV["LINES"]=100
# disable truncation of dataframes columns
import Base.show
show(df::AbstractDataFrame) = show(df,truncate=0)
show(io::IO,df::AbstractDataFrame) = show(io,df;truncate=0)

show (generic function with 340 methods)

In [3]:
# brief overview of AutoMLPipeline package
# more details is in my Juliacon2020 presentation about this package
# we use AutoMLPipeline to develop the pipeline optimizer

# define scalers
rb     = SKPreprocessor("RobustScaler",Dict(:name=>"rb"))
pt     = SKPreprocessor("PowerTransformer",Dict(:name=>"pt"))
norm   = SKPreprocessor("Normalizer",Dict(:name=>"norm"))
mx     = SKPreprocessor("MinMaxScaler",Dict(:name=>"mx"))
std    = SKPreprocessor("StandardScaler",Dict(:name=>"std"))
# define extractors
pca    = SKPreprocessor("PCA",Dict(:name=>"pca"))
fa     = SKPreprocessor("FactorAnalysis",Dict(:name=>"fa"))
ica    = SKPreprocessor("FastICA",Dict(:name=>"ica"))
# define learners
rf     = SKLearner("RandomForestClassifier",Dict(:name => "rf"))
ada    = SKLearner("AdaBoostClassifier",Dict(:name => "ada"))
gb     = SKLearner("GradientBoostingClassifier",Dict(:name => "gb"))
lsvc   = SKLearner("LinearSVC",Dict(:name => "lsvc"))
rbfsvc = SKLearner("SVC",Dict(:name => "rbfsvc"))
dt     = SKLearner("DecisionTreeClassifier",Dict(:name =>"dt"))
# preprocessing
noop = Identity(Dict(:name =>"noop"))
ohe  = OneHotEncoder(Dict(:name=>"ohe"))
catf = CatFeatureSelector(Dict(:name=>"catf"))
numf = NumFeatureSelector(Dict(:name=>"numf"))

vscalers =    [rb,pt,norm,mx,std,noop]
vextractors = [pca,fa,ica,noop]
vlearners =   [rf,gb,lsvc,rbfsvc,ada,dt]

dataset = getiris()
X = dataset[:,1:4]
Y = dataset[:,5] |> collect;

In [4]:
# one-block pipeline
prep = @pipeline (catf |> ohe) + numf
ppl1 = @pipeline prep |> (mx |> pca) |> rf
crossvalidate(ppl1,X,Y)

fold: 1, 86.66666666666667
fold: 2, 86.66666666666667
fold: 3, 86.66666666666667
fold: 4, 100.0
fold: 5, 100.0
fold: 6, 100.0
fold: 7, 100.0
fold: 8, 100.0
fold: 9, 93.33333333333333
fold: 10, 100.0
errors: 0


(mean = 95.33333333333334, std = 6.324555320336757, folds = 10, errors = 0)

In [5]:
# two-block pipeline
ppl2 = @pipeline prep |> ((norm |> ica) + (pt |> pca)) |> lsvc
crossvalidate(ppl2,X,Y)

fold: 1, 100.0
fold: 2, 100.0
fold: 3, 93.33333333333333
fold: 4, 93.33333333333333
fold: 5, 100.0
fold: 6, 93.33333333333333
fold: 7, 100.0
fold: 8, 100.0
fold: 9, 100.0
fold: 10, 100.0
errors: 0


(mean = 98.0, std = 3.2203059435976553, folds = 10, errors = 0)

In [6]:
function oneblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt in extractors
         @distributed (vcat) for sc in scalers
            # baseline preprocessing
            prep = @pipeline ((catf |> ohe) + numf)
            # one-block prp
            expx = @pipeline prep |> (sc |> xt) |> lr
            scn   = sc.name[1:end - 4];xtn = xt.name[1:end - 4]; lrn = lr.name[1:end - 4]
            pname = "($scn |> $xtn) |> $lrn"
            DataFrame(Description=pname,Pipeline=expx)
         end
      end
   end
   return results
end

oneblock_pipeline_factory (generic function with 1 method)

In [7]:
res_oneblock = oneblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_BYf"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_BYf"", :machines=>Machine[Pipeline(""linearpipeline_mP4"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_mP4"", :machines=>Workflow[ComboPipeline(""combopipeline_LcV"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_LcV"", :machines=>Machine[Pipeline(""linearpipeline_TIl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_TIl"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_PVw"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_PVw"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_Uih"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_Uih"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(pt |> pca) |> rf,"Pipeline(""linearpipeline_u4V"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_u4V"", :machines=>Machine[Pipeline(""linearpipeline_o8s"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_o8s"", :machines=>Workflow[ComboPipeline(""combopipeline_EqM"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_EqM"", :machines=>Machine[Pipeline(""linearpipeline_5BL"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_5BL"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_WO0"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_WO0"", :machines=>SKPreprocessor[SKPreprocessor(""pt_uIA"", Dict{Symbol, Any}(:name=>""pt_uIA"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_Uih"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_Uih"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
3,(norm |> pca) |> rf,"Pipeline(""linearpipeline_h8G"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_h8G"", :machines=>Machine[Pipeline(""linearpipeline_6H1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_6H1"", :machines=>Workflow[ComboPipeline(""combopipeline_Sfn"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Sfn"", :machines=>Machine[Pipeline(""linearpipeline_bi5"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_bi5"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), 

In [8]:
function evaluate_pipeline(dfpipelines,X,Y;folds=3)
   res=@distributed (vcat) for prow in eachrow(dfpipelines)
      perf = crossvalidate(prow.Pipeline,X,Y;nfolds=folds)
      DataFrame(Description=prow.Description,mean=perf.mean,sd=perf.std)
   end
   return res
end

evaluate_pipeline (generic function with 1 method)

In [9]:
cross_res_oneblock = evaluate_pipeline(res_oneblock,X,Y)

      From worker 6:	fold: 1, 98.0
      From worker 7:	fold: 1, 94.0
      From worker 7:	fold: 2, 92.0
      From worker 6:	fold: 2, 98.0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From worker 6:	fold: 3, 90.0
      From worker 6:	errors: 0
      From worker 5:	fold: 1, 88.0
      From worker 7:	fold: 1, 98.0
      From worker 6:	fold: 1, 96.0
      From worker 7:	fold: 2, 98.0
      From worker 5:	fold: 2, 86.0
      From worker 6:	fold: 2, 92.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 7:	fold: 3, 92.0
      From worker 7:	errors: 0
      From worker 6:	fold: 3, 100.0
      From worker 6:	errors: 0
      From worker 5:	fold: 1, 82.0
      From worker 5:	fold: 2, 84.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 3:	fold: 1, 94.0
      From worker 7:	fold: 1, 98.0
      From worker 7:	fold: 2, 98.0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From w

      From worker 4:	fold: 3, 94.0
      From worker 4:	errors: 0
      From worker 7:	fold: 3, 68.0
      From worker 7:	errors: 0
      From worker 2:	fold: 1, 92.0
      From worker 7:	fold: 1, 92.0
      From worker 3:	fold: 2, 96.0
      From worker 9:	fold: 1, 92.0
      From worker 9:	fold: 2, 98.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 7:	fold: 2, 94.0
      From worker 2:	fold: 2, 98.0
      From worker 8:	fold: 1, 94.0
      From worker 4:	fold: 1, 92.0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 9:	fold: 1, 90.0
      From worker 9:	fold: 2, 92.0
      From worker 9:	fold: 3, 94.0
      From worker 9:	errors: 0
      From worker 8:	fold: 2, 94.0
      From worker 7:	fold: 1, 78.0
      From worker 2:	fold: 3, 88.0
      From worker 2:	errors: 0
      From worker 8:	fold: 3, 98.0
      From worker 8:	errors: 0
      From worker

      From worker 4:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 2:	fold: 2, 96.0
      From worker 4:	fold: 1, 94.0
      From worker 3:	fold: 1, 90.0
      From worker 2:	fold: 3, 100.0
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 96.0
      From worker 4:	fold: 2, 98.0
      From worker 3:	fold: 2, 96.0
      From worker 2:	fold: 2, 94.0
      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 3:	fold: 3, 100.0
      From worker 3:	errors: 0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 4:	fold: 1, 96.0
      From worker 3:	fold: 1, 92.0
      From worker 2:	fold: 1, 96.0
      From worker 4:	fold: 2, 98.0
      From worker 3:	fold: 2, 94.0
      From worker 2:	fold: 2, 92.0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 4:	fold: 3, 94.0
      From worker 4:	errors: 0
      From work

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rf,88.6667,7.02377
2,(pt |> pca) |> rf,92.6667,3.05505
3,(norm |> pca) |> rf,98.0,0.0
4,(mx |> pca) |> rf,93.3333,3.05505
5,(std |> pca) |> rf,92.6667,5.03322
6,(noop |> pca) |> rf,94.6667,4.16333
7,(rb |> fa) |> rf,88.0,2.0
8,(pt |> fa) |> rf,92.6667,3.05505
9,(norm |> fa) |> rf,93.3333,2.3094


In [10]:
sort!(cross_res_oneblock,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(norm |> pca) |> rf,98.0,0.0
2,(rb |> ica) |> rf,98.0,2.0
3,(rb |> ica) |> gb,98.0,2.0
4,(norm |> pca) |> rbfsvc,98.0,0.0
5,(norm |> ica) |> rbfsvc,98.0,2.0
6,(norm |> pca) |> dt,98.0,2.0
7,(noop |> ica) |> rf,97.3333,3.05505
8,(norm |> pca) |> gb,97.3333,3.05505
9,(std |> pca) |> gb,97.3333,1.1547


In [11]:
function twoblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt1 in extractors
         @distributed (vcat) for xt2 in extractors
            @distributed (vcat) for sc1 in scalers
               @distributed (vcat) for sc2 in scalers
                  prep = @pipeline ((catf |> ohe) + numf)
                  expx = @pipeline prep |> ((sc1 |> xt1) + (sc2 |> xt2)) |> lr
                  scn1   = sc1.name[1:end - 4];xtn1 = xt1.name[1:end - 4];
                  scn2   = sc2.name[1:end - 4];xtn2 = xt2.name[1:end - 4];
                  lrn = lr.name[1:end - 4]
                  pname = "($scn1 |> $xtn1) + ($scn2 |> $xtn2) |> $lrn"
                  DataFrame(Description=pname,Pipeline=expx)
               end
            end
         end
      end
   end
   return results
end
res_twoblock = twoblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rf,"Pipeline(""linearpipeline_0K1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_0K1"", :machines=>Machine[Pipeline(""linearpipeline_Fqf"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Fqf"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_INZ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_INZ"", :machines=>Machine[Pipeline(""linearpipeline_r4X"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_r4X"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_qbY"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_qbY"", :machines=>Pipeline[Pipeline(""linearpipeline_6eZ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_6eZ"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_6Ps"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_6Ps"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rf_Uih"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_Uih"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) + (pt |> pca) |> rf,"Pipeline(""linearpipeline_P9Q"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_P9Q"", :machines=>Machine[Pipeline(""linearpipeline_Rqw"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Rqw"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_ePx"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_ePx"", :machines=>Machine[Pipeline(""linearpipeline_2rC"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_2rC"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_K42"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_K42"", :machines=>Pipeline[Pipeline(""linearpipeline_iHc"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_iHc"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_jOa"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_jOa"", :machines=>SKPreprocessor[SKPreprocessor(""pt_uIA"", Dict{Symbol, Any}(:name=>""pt_uIA"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocesso

In [12]:
function model_selection_pipeline(learners)
   results = @distributed (vcat) for lr in learners
      prep = @pipeline ((catf |> ohe) + numf)
      expx = @pipeline prep |> (rb |> pca)  |> lr
      pname = "(rb |> pca) |> $(lr.name[1:end-4])"
      DataFrame(Description=pname,Pipeline=expx)
   end
   return results
end
dfpipes = model_selection_pipeline(vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_Bqc"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Bqc"", :machines=>Machine[Pipeline(""linearpipeline_Ghe"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Ghe"", :machines=>Workflow[ComboPipeline(""combopipeline_3XJ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_3XJ"", :machines=>Machine[Pipeline(""linearpipeline_RK4"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_RK4"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_vL3"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_vL3"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_Uih"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_Uih"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) |> gb,"Pipeline(""linearpipeline_NBr"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_NBr"", :machines=>Machine[Pipeline(""linearpipeline_Uc8"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Uc8"", :machines=>Workflow[ComboPipeline(""combopipeline_Ssf"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Ssf"", :machines=>Machine[Pipeline(""linearpipeline_rPM"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_rPM"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_HQV"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_HQV"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""gb_Ikz"", Dict{Symbol, Any}(:output=>:class, :name=>""gb_Ikz"", :impl_args=>Dict{Symbol, Any}(), :learner=>""GradientBoostingClassifier""))]))"
3,(rb |> pca) |> lsvc,"Pipeline(""linearpipeline_bn2"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_bn2"", :machines=>Machine[Pipeline(""linearpipeline_4Y9"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_4Y9"", :machines=>Workflow[ComboPipeline(""combopipeline_06K"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_06K"", :machines=>Machine[Pipeline(""linearpipeline_9TZ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9TZ"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), 

In [13]:
# find the best model
@everywhere Random.seed!(10)
res = evaluate_pipeline(dfpipes,X,Y;folds=10)
sort!(res,:mean, rev = true)

      From worker 5:	fold: 1, 86.66666666666667
      From worker 4:	fold: 1, 86.66666666666667
      From worker 7:	fold: 1, 86.66666666666667
      From worker 5:	fold: 2, 100.0
      From worker 4:	fold: 2, 86.66666666666667
      From worker 7:	fold: 2, 100.0
      From worker 5:	fold: 3, 93.33333333333333
      From worker 4:	fold: 3, 100.0
      From worker 7:	fold: 3, 86.66666666666667
      From worker 5:	fold: 4, 100.0
      From worker 4:	fold: 4, 100.0
      From worker 7:	fold: 4, 86.66666666666667
      From worker 5:	fold: 5, 100.0
      From worker 4:	fold: 5, 86.66666666666667
      From worker 7:	fold: 5, 93.33333333333333
      From worker 5:	fold: 6, 100.0
      From worker 7:	fold: 6, 100.0
      From worker 4:	fold: 6, 100.0
      From worker 5:	fold: 7, 93.33333333333333
      From worker 7:	fold: 7, 93.33333333333333
      From worker 4:	fold: 7, 93.33333333333333
      From worker 5:	fold: 8, 100.0
      From worker 7:	fold: 8, 86.66666666666667
      From worke

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rbfsvc,96.0,4.66137
2,(rb |> pca) |> lsvc,94.0,5.8373
3,(rb |> pca) |> rf,93.3333,7.02728
4,(rb |> pca) |> gb,92.6667,3.78431
5,(rb |> pca) |> dt,92.0,5.25874
6,(rb |> pca) |> ada,74.0,12.7463


In [14]:
# use the best model to generate pipeline search
dfp = twoblock_pipeline_factory(vscalers,vextractors,[rbfsvc])

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rbfsvc,"Pipeline(""linearpipeline_dfK"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_dfK"", :machines=>Machine[Pipeline(""linearpipeline_p5S"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_p5S"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_i5g"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_i5g"", :machines=>Machine[Pipeline(""linearpipeline_9ls"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9ls"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_hLt"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_hLt"", :machines=>Pipeline[Pipeline(""linearpipeline_BMl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_BMl"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_KDj"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_KDj"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rbfsvc_u0e"", Dict{Symbol, Any}(:output=>:class, :name=>""rbfsvc_u0e"", :impl_args=>Dict{Symbol, Any}(), :learner=>""SVC""))]))"
2,(rb |> pca) + (pt |> pca) |> rbfsvc,"Pipeline(""linearpipeline_Tob"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Tob"", :machines=>Machine[Pipeline(""linearpipeline_wBe"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_wBe"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_3Cw"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_3Cw"", :machines=>Machine[Pipeline(""linearpipeline_BFu"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_BFu"", :machines=>Transformer[CatFeatureSelector(""catf_Rx4"", Dict{Symbol, Any}(:name=>""catf_Rx4"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_heO"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_heO"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_qXY"", Dict{Symbol, Any}(:name=>""numf_qXY"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_Zmz"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Zmz"", :machines=>Pipeline[Pipeline(""linearpipeline_zfE"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_zfE"", :machines=>SKPreprocessor[SKPreprocessor(""rb_coq"", Dict{Symbol, Any}(:name=>""rb_coq"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_IR3"", Dict{Symbol, Any}(:name=>""pca_IR3"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_8vY"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_8vY"", :machines=>SKPreprocessor[SKPreprocessor(""pt_uIA"", Dict{Symbol, Any}(:name=>""pt_uIA"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor("

In [16]:
# evaluate the pipeline
@everywhere Random.seed!(10)
bestp=evaluate_pipeline(dfp,X,Y;folds=3)

      From worker 2:	fold: 1, 94.0
      From worker 2:	fold: 2, 94.0
      From worker 4:	fold: 1, 94.0
      From worker 5:	fold: 1, 96.0
      From worker 6:	fold: 1, 94.0
      From worker 2:	fold: 3, 96.0
      From worker 2:	errors: 0
      From worker 4:	fold: 2, 92.0
      From worker 7:	fold: 1, 82.0
      From worker 9:	fold: 1, 94.0
      From worker 5:	fold: 2, 94.0
      From worker 6:	fold: 2, 96.0
      From worker 4:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 7:	fold: 2, 94.0
      From worker 9:	fold: 2, 96.0
      From worker 2:	fold: 1, 100.0
      From worker 4:	fold: 1, 100.0
      From worker 2:	fold: 2, 96.0
      From worker 2:	fold: 3, 92.0
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 2:	fold: 2, 94.0
      From worker 5:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 3:	fold: 1, 94.0
      From worker 2:	fold: 3, 94.0
   

      From worker 6:	/Users/ppalmes/.local/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn('FastICA did not converge. Consider increasing '
      From worker 6:	/Users/ppalmes/.local/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn('FastICA did not converge. Consider increasing '
      From worker 6:	/Users/ppalmes/.local/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
      From worker 6:	  warnings.warn('FastICA did not converge. Consider increasing '
      From worker 6:	/Users/ppalmes/.local/lib/python3.8/site-packages/sk

      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 8:	fold: 2, 96.0
      From worker 4:	fold: 3, 100.0
      From worker 2:	fold: 1, 94.0
      From worker 4:	errors: 0
      From worker 9:	fold: 1, 96.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 96.0
      From worker 4:	fold: 1, 96.0
      From worker 6:	fold: 1, 96.0
      From worker 2:	fold: 2, 96.0
      From worker 8:	fold: 2, 96.0
      From worker 7:	fold: 2, 94.0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 8:	fold: 3, 98.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 96.0
      From worker 4:	fold: 2, 96.0
      From worker 2:	fold: 3, 96.0
      From worker 2:	errors: 0
      From worker 8:	fold: 2, 96.0
      From worker 5:	fold: 1, 94.0
      From worker 2:	fold: 1, 96.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 9:	fold: 2, 92.0
      From w

      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 9:	fold: 3, 100.0
      From worker 2:	fold: 3, 98.0
      From worker 3:	fold: 2, 96.0
      From worker 9:	errors: 0
      From worker 2:	errors: 0
      From worker 8:	fold: 1, 96.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 2:	fold: 1, 94.0
      From worker 4:	fold: 1, 98.0
      From worker 6:	fold: 2, 96.0
      From worker 4:	fold: 2, 100.0
      From worker 2:	fold: 2, 96.0
      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 8:	fold: 2, 92.0
      From worker 3:	errors: 0
      From worker 2:	fold: 3, 98.0
      From worker 2:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 7:	fold: 1, 88.0
      From worker 4:	fold: 1, 94.0
      From worker 8:	fold: 3, 98.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 98.0
      From worker 2:	fold: 1, 96.0
      From work

      From worker 7:	fold: 2, 90.0
      From worker 3:	fold: 1, 98.0
      From worker 2:	fold: 2, 94.0
      From worker 4:	fold: 1, 92.0
      From worker 6:	fold: 2, 100.0
      From worker 3:	fold: 2, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 8:	fold: 1, 98.0
      From worker 6:	errors: 0
      From worker 2:	fold: 3, 94.0
      From worker 4:	fold: 2, 96.0
      From worker 2:	errors: 0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 6:	fold: 1, 94.0
      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 2:	fold: 1, 98.0
      From worker 8:	fold: 2, 96.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 6:	fold: 2, 98.0
      From worker 4:	fold: 1, 96.0
      From worker 9:	fold: 2, 98.0
      From worker 2:	fold: 2, 90.0
      From worker 6:	fold: 3, 96.0
      From worker 4:	fold: 2, 94.0
      From worker 9:	fold: 3, 90.0
      From worker 6:	errors: 0
      Fr

...

      From worker 3:	fold: 2, 100.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 7:	fold: 2, 88.0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 92.0
      From worker 3:	fold: 2, 96.0
      From worker 3:	fold: 3, 100.0
      From worker 7:	fold: 3, 90.0
      From worker 3:	errors: 0
      From worker 7:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 3:	fold: 1, 98.0
      From worker 7:	fold: 1, 92.0
      From worker 3:	fold: 2, 94.0
      From worker 5:	fold: 2, 98.0
      From worker 6:	fold: 3, 94.0
      From worker 6:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 7:	fold: 2, 88.0
      From worker 5:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 3:	fold: 1, 98.0
      From worker 3:	fold: 2, 94.0
      From worker 5:	fold: 1, 98.0
      From worker 3:	fold: 3, 94.0
      From worker 3:	errors: 0
      From work

      From worker 7:	fold: 3, 98.0
      From worker 7:	errors: 0
      From worker 3:	fold: 3, 94.0
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 96.0
      From worker 5:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 3:	fold: 2, 92.0
      From worker 5:	fold: 1, 96.0
      From worker 7:	fold: 1, 92.0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 94.0
      From worker 5:	fold: 2, 96.0
      From worker 7:	fold: 2, 94.0
      From worker 3:	fold: 2, 100.0
      From worker 5:	fold: 3, 100.0
      From worker 5:	errors: 0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 6:	fold: 2, 98.0
      From worker 3:	fold: 1, 96.0
      From worker 3:	fold: 2, 96.0
      From worker 3:	fold: 3, 98.0
      From worker 3:	errors: 0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 3:	fold: 1, 94.0
      From work

      From worker 7:	fold: 2, 96.0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 6:	fold: 3, 98.0
      From worker 7:	fold: 1, 98.0
      From worker 6:	errors: 0
      From worker 6:	fold: 1, 98.0
      From worker 7:	fold: 2, 98.0
      From worker 6:	fold: 2, 96.0
      From worker 6:	fold: 3, 100.0
      From worker 6:	errors: 0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 6:	fold: 1, 98.0
      From worker 6:	fold: 2, 92.0
      From worker 7:	fold: 1, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 6:	fold: 1, 96.0
      From worker 7:	fold: 2, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 6:	fold: 3, 98.0
      From worker 6:	errors: 0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 96.0
      From worker 6:	fold: 1, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 6:	fold: 3, 94.0
      From w

      From worker 7:	fold: 1, 100.0
      From worker 7:	fold: 2, 98.0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 96.0
      From worker 7:	fold: 2, 94.0
      From worker 7:	fold: 3, 100.0
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 96.0
      From worker 7:	fold: 2, 96.0
      From worker 7:	fold: 3, 92.0
      From worker 7:	errors: 0


,Description,mean,sd
,String,Float64,Float64
1,(noop |> pca) + (mx |> noop) |> rbfsvc,99.3333,1.1547
2,(noop |> pca) + (noop |> ica) |> rbfsvc,98.6667,1.1547
3,(norm |> fa) + (rb |> pca) |> rbfsvc,98.6667,1.1547
4,(norm |> fa) + (mx |> pca) |> rbfsvc,98.6667,1.1547
5,(mx |> fa) + (norm |> fa) |> rbfsvc,98.6667,2.3094
6,(std |> ica) + (norm |> pca) |> rbfsvc,98.6667,1.1547
7,(std |> ica) + (noop |> pca) |> rbfsvc,98.6667,2.3094
8,(noop |> ica) + (noop |> pca) |> rbfsvc,98.6667,1.1547
9,(mx |> ica) + (norm |> fa) |> rbfsvc,98.6667,1.1547


In [18]:
sort!(bestp,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(noop |> pca) + (mx |> noop) |> rbfsvc,99.3333,1.1547
2,(noop |> pca) + (noop |> ica) |> rbfsvc,98.6667,1.1547
3,(norm |> fa) + (rb |> pca) |> rbfsvc,98.6667,1.1547
4,(norm |> fa) + (mx |> pca) |> rbfsvc,98.6667,1.1547
5,(mx |> fa) + (norm |> fa) |> rbfsvc,98.6667,2.3094
6,(std |> ica) + (norm |> pca) |> rbfsvc,98.6667,1.1547
7,(std |> ica) + (noop |> pca) |> rbfsvc,98.6667,2.3094
8,(noop |> ica) + (noop |> pca) |> rbfsvc,98.6667,1.1547
9,(mx |> ica) + (norm |> fa) |> rbfsvc,98.6667,1.1547


In [19]:
show(bestp;allrows=true,truncate=0,allcols=true)

576×3 DataFrame
 Row │ Description                                mean     sd      
     │ String                                     Float64  Float64 
─────┼─────────────────────────────────────────────────────────────
   1 │ (noop |> pca) + (mx |> noop) |> rbfsvc     99.3333  1.1547
   2 │ (noop |> pca) + (noop |> ica) |> rbfsvc    98.6667  1.1547
   3 │ (norm |> fa) + (rb |> pca) |> rbfsvc       98.6667  1.1547
   4 │ (norm |> fa) + (mx |> pca) |> rbfsvc       98.6667  1.1547
   5 │ (mx |> fa) + (norm |> fa) |> rbfsvc        98.6667  2.3094
   6 │ (std |> ica) + (norm |> pca) |> rbfsvc     98.6667  1.1547
   7 │ (std |> ica) + (noop |> pca) |> rbfsvc     98.6667  2.3094
   8 │ (noop |> ica) + (noop |> pca) |> rbfsvc    98.6667  1.1547
   9 │ (mx |> ica) + (norm |> fa) |> rbfsvc       98.6667  1.1547
  10 │ (noop |> pca) + (norm |> pca) |> rbfsvc    98.0     0.0
  11 │ (noop |> pca) + (mx |> pca) |> rbfsvc      98.0     2.0
  12 │ (noop |> pca) + (noop |> pca) |> rbfsvc    98.0     0